BRI - 33979

In [1]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [2]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [ ]:
searchTerm1 = ['BIM', 'Building Information Modelling']
searchTerm2 = ['Green Building', 'Green Buildings']

tc = 0

# searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
# searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

# https://www.tandfonline.com/loi/rbri20
    
# https://www.tandfonline.com/action/doSearch?AllField=BIM+and+Green+Building&SeriesKey=rbri20
    
baseUrl = 'https://www.tandfonline.com/action/doSearch?AllField='
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        
        query = baseUrl + item1 +"+AND+"+ item2+"&SeriesKey=rbri20"
#         print(query)

        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
#         print(number)
#         tc = tc + number
# print("Final: ",tc)
        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
        print(pageSize)

        cnt = 0
        allLinks = []
        
#         for testing purpose only
#         pageSize =1 
        for currentPage in range(0,pageSize):
            print("In page: ",currentPage)
            
            if currentPage != 0:
                newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                cnt = cnt + 1
            else:
                newQuery = query
            
            print("Searching for query: ",newQuery)
            response = requests.get(newQuery)
            soup = BeautifulSoup(response.text, 'html.parser')
            mydivs = soup.findAll("a", {'class':['ref nowrap']})
            
            for d in mydivs:
                url = 'https://www.tandfonline.com' + d.get('href')
                allLinks.append(url)

        for link in allLinks:
            df.loc[row,'searchTerm1'] = item1
            df.loc[row,'searchTerm2'] = item2
            df.loc[row,'url'] = link
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "lxml")
            
            for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
                print("Title: ",i)
                df.loc[row,'Title'] = i
            
            temp = soup.findAll('h2',{'class':['section-heading-2']} )
            print("Number of sections: ",len(temp))            
            
            count = 0
            for i in range(len(temp)):

                if temp[i].text == temp[i].find_next('p').text:
        
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)                    
                    df.loc[row,sectionData] = extractData
                    count = count + 1
                else:
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)
                    df.loc[row,sectionData] = extractData
                    count = count + 1
            
            
            row = row + 1


['Green+Building', 'Green+Buildings']
['BIM', 'Building+Information+Modelling']
7
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=rbri20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=4
In page:  5
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=rbri20&page

Title:  Service life prediction and environmental exposure conditions of timber claddings in South Chile
Number of sections:  11
7
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=rbri20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=4
In page:  5
Searching for query:  https://www.tandfonline.com/action/doSearch

Title:  Index, Volume 45
Number of sections:  2
Title:  Disruptive information exchange requirements in construction projects: perception and response patterns
Number of sections:  8
126
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllFi

In page:  42
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=42
In page:  43
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=43
In page:  44
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=44
In page:  45
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=45
In page:  46
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=46
In page:  47
Searching for que

In page:  85
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=85
In page:  86
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=86
In page:  87
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=87
In page:  88
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=88
In page:  89
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Building&SeriesKey=rbri20&pageSize=10&subjectTitle=&startPage=89
In page:  90
Searching for que

...

Title:  Revisiting thermal comfort models in Iranian classrooms during the warm season
Number of sections:  6
Title:  What is wrong with energy efficiency?
Number of sections:  9
Title:  Responsible property investment criteria developed using the Delphi Method
Number of sections:  7
Title:  Incorporating inhabitants’ everyday practices into domestic retrofits
Number of sections:  8
Title:  Index, Volume 45
Number of sections:  2
Title:  Developing English domestic occupancy profiles
Number of sections:  7
Title:  Energy performance gaps: promises, people, practices
Number of sections:  8
Title:  Cost and value ratios of operating renewed English secondary schools
Number of sections:  11
Title:  Towards a research agenda for adapting to climate change
Number of sections:  0
Title:  Passive downdraught evaporative cooling: performance in a prototype house
Number of sections:  8
Title:  Hygrothermal assessment of a traditional earthen wall in a dry Mediterranean climate
Number of section

Title:  Heatwave planning: community involvement in co-producing resilience
Number of sections:  12
Title:  Sustainability and risk premium estimation in property valuation and assessment of worth
Number of sections:  8
Title:  A practice–theory approach to homeowners' energy retrofits in four European areas
Number of sections:  9
Title:  Maintenance in hospitals with less than 200 beds: efficiency indicators
Number of sections:  7
Title:  The impact of BRE's commercialization on the research community
Number of sections:  0
Title:  Really rethinking construction
Number of sections:  0
Title:  Redefining ICT embeddedness in the construction industry: maximizing technology diffusion capabilities to support agility
Number of sections:  7
Title:  Historical energy analysis of the Norwegian dwelling stock
Number of sections:  5
Title:  Making energy visible in domestic property markets: the influence of advertisements
Number of sections:  9
Title:  An intuitive daylighting performance anal

Title:  Ventilation strategies for urban housing: lessons from a PoE case study
Number of sections:  12
Title:  Developing a multicriteria approach for the measurement of sustainable performance
Number of sections:  6
Title:  Temperature and adaptive comfort in heated, cooled and free-running dwellings
Number of sections:  8
Title:  Thermal seasonal variation and occupants’ spatial behaviour in domestic spaces
Number of sections:  8
Title:  Ecological resilience and resilient cities
Number of sections:  9
Title:  Application of an alternative contractor selection model
Number of sections:  0
Title:  Inspiring low-energy retrofits: the influence of ‘open home’ events
Number of sections:  6
Title:  Nostalgic research reflections
Number of sections:  0
Title:  Design quality in pre-project planning: applications of the Project Definition Rating Index
Number of sections:  0
Title:  Housing renovations and energy efficiency: insights from homeowners’ practices
Number of sections:  7
Title: 

Title:  Dwelling performance and adaptive summer comfort in low-income Australian households
Number of sections:  9
Title:  Added value of good design
Number of sections:  13
Title:  Investigating the indoor environmental quality of different workplaces through web-scraping and text-mining of Glassdoor reviews
Number of sections:  7
Title:  Priority setting in planned maintenance - practical issues in using the multi-attribute approach
Number of sections:  0
Title:  Implications of climate change on heritage
Number of sections:  9
Title:  Relationship between moisture content and strength of soilcrete blocks
Number of sections:  0
Title:  Integrated framework of home comfort: relaxation, companionship and control
Number of sections:  9
Title:  Computer-aided qualitative data analysis: panacea or paradox? 
Number of sections:  7
Title:  A theoretical model for determining equipment service life
Number of sections:  0
Title:  Understanding investment drivers for UK sustainable property
N

Title:  The economics of ice thermal storage
Number of sections:  0
Title:  Towards a holistic approach for assessing the impact of IEQ on satisfaction, health, and productivity
Number of sections:  9
Title:  Organizational memory formation and its use in construction
Number of sections:  7
Title:  Analysing architecture through drawing
Number of sections:  9
Title:  On the discourse of construction competitiveness
Number of sections:  9
Title:  Competitiveness assessment system for China's construction industry
Number of sections:  7
Title:  Rethinking thermal comfort
Number of sections:  1
Title:  Thermal comfort in naturally ventilated primary school classrooms
Number of sections:  7
Title:  Moisture permeability data presented as a mathematical relationship
Number of sections:  0
Title:  Collaborative knowledge work environments
Number of sections:  11
Title:  Market value of sustainability business innovations in the construction sector
Number of sections:  11
Title:  Towards a kn

In [ ]:
df.iloc[:, 0:36]

In [ ]:
df = df.where(pd.notnull(df), None)
df


In [ ]:
pwd

In [ ]:
cd Desktop/MDSOutput

In [ ]:
df.to_csv('KG-BRI.csv')